In [7]:
!pip install gensim

You should consider upgrading via the '/Users/max/.pyenv/versions/3.7.11/bin/python3.7 -m pip install --upgrade pip' command.


In [8]:
!pip install tensorflow

You should consider upgrading via the '/Users/max/.pyenv/versions/3.7.11/bin/python3.7 -m pip install --upgrade pip' command.


In [9]:
!pip install tensorflow-hub

You should consider upgrading via the '/Users/max/.pyenv/versions/3.7.11/bin/python3.7 -m pip install --upgrade pip' command.


In [10]:
!pip install nltk

You should consider upgrading via the '/Users/max/.pyenv/versions/3.7.11/bin/python3.7 -m pip install --upgrade pip' command.


In [35]:
!pip install sentence-transformers

     |████████████████████████████████| 79 kB 3.0 MB/s eta 0:00:01
     |████████████████████████████████| 3.8 MB 9.1 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 36.6 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 32.6 MB/s eta 0:00:01
     |████████████████████████████████| 67 kB 8.7 MB/s  eta 0:00:01
  Using cached packaging-21.3-py3-none-any.whl (40 kB)
     |████████████████████████████████| 3.6 MB 24.6 MB/s eta 0:00:01
  Using cached PyYAML-6.0-cp37-cp37m-macosx_10_9_x86_64.whl (189 kB)
     |████████████████████████████████| 895 kB 43.9 MB/s eta 0:00:01
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=090e0ee5ed789b85b8841e4d3ecbb142cebb77ea53a79d46c7b436a3637fc222
  Stored in directory: /Users/max/Library/Caches/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
ERROR: torchvision 0.12.0 has requirement torch==1.11.0

In [36]:
import pandas as pd
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import tensorflow_hub as hub
from nltk.tokenize import word_tokenize
from scipy import spatial
import nltk
from sentence_transformers import SentenceTransformer
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/max/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

here we generate the dataframe, which keyword named as title and GT recommendation named as content.

We are using here just the NYT articles, as buy using articles from just one publisher we hope to get a better more consistant matching and less confusion, which is important, when building the pipeline

In [26]:
df = pd.read_csv('../comparison_dataset/articles1.csv')
df = df[df['publication'] == 'New York Times']
df = df.drop(columns = ['id', 'publication', 'author', 'date', 'year', 'month', 'url', 'Unnamed: 0'])
df

,title,content
0,House Republicans Fret About Winning Their Hea...,WASHINGTON — Congressional Republicans have...
1,Rift Between Officers and Residents as Killing...,"After the bullet shells get counted, the blood..."
2,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...","When Walt Disney’s “Bambi” opened in 1942, cri..."
3,"Among Deaths in 2016, a Heavy Toll in Pop Musi...","Death may be the great equalizer, but it isn’t..."
4,Kim Jong-un Says North Korea Is Preparing to T...,"SEOUL, South Korea — North Korea’s leader, ..."
...,...,...
7798,U.N. Relief Official Calls Crisis in Aleppo th...,The top aid official at the United Nations gav...
7799,Federal Judge Curbs Enforcement of North Carol...,A federal judge on Friday curbed the enforceme...
7800,Mexicans Accuse President of ‘Historic Error’ ...,MEXICO CITY — If President Enrique Peña Nie...
7801,"U.S. Presses for Truce in Syria, With Its Larg...","HANGZHOU, China — The image of a Syrian ..."


now we try to find embedding methods, which in an ideal matching, or we could also just try to minimize distance between the embedded title and content

In [27]:
def embedding_by_doc_2_vec(df):
  data = df['content']
  tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data)]
  # df['tagged_data'] = tagged_data
  max_epochs = 5
  vec_size = 20
  alpha = 0.025

  model = Doc2Vec(#size = vec_size,
    vector_size=vec_size,
                  alpha=alpha, 
                  min_alpha=0.00025,
                  min_count=1,
                  dm =1)
    
  model.build_vocab(tagged_data)

  for epoch in range(max_epochs):
      print('iteration {0}'.format(epoch))
      model.train(tagged_data,
                  total_examples=model.corpus_count,
                  #epochs = model.iter
                  epochs=model.epochs)
      # decrease the learning rate
      model.alpha -= 0.0002
      # fix the learning rate, no decay
      model.min_alpha = model.alpha
  print('here')
  print(model.docvecs['1'])
  return model

In [28]:
#model_d2v = embedding_by_doc_2_vec(df)

In [29]:
#model_d2v.save("doc2vec.model")
model_d2v = Doc2Vec.load("doc2vec.model")

In [30]:
def embedd_d2v(model, doc):
    return model.infer_vector(doc.split())

df['embed_title_d2v'] = df['title'].apply(lambda x: embedd_d2v(model_d2v, x))
df['embed_content_d2v'] = df['content'].apply(lambda x: embedd_d2v(model_d2v, x))
df

,title,content,embed_title_d2v,embed_content_d2v
0,House Republicans Fret About Winning Their Hea...,WASHINGTON — Congressional Republicans have...,"[0.0097255735, 0.0013119221, 0.0031621992, -0....","[-0.4876673, -0.03852097, -0.8450327, 5.006812..."
1,Rift Between Officers and Residents as Killing...,"After the bullet shells get counted, the blood...","[-0.038155433, 0.32817695, 0.029026736, -0.019...","[3.0910404, 0.4343017, -0.09805204, 0.16859086..."
2,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...","When Walt Disney’s “Bambi” opened in 1942, cri...","[-0.06825161, 0.5558198, 0.057838302, -0.01888...","[1.2336452, 2.1413794, 2.6180854, -3.2745428, ..."
3,"Among Deaths in 2016, a Heavy Toll in Pop Musi...","Death may be the great equalizer, but it isn’t...","[-0.06487101, 0.12077401, 0.030435491, 0.12945...","[-1.9785938, 2.9619815, 6.096006, -0.8292187, ..."
4,Kim Jong-un Says North Korea Is Preparing to T...,"SEOUL, South Korea — North Korea’s leader, ...","[-0.05594013, 0.07169301, 0.031006414, -0.0133...","[0.68977714, 3.0600314, 3.0057077, 2.6856246, ..."
...,...,...,...,...
7798,U.N. Relief Official Calls Crisis in Aleppo th...,The top aid official at the United Nations gav...,"[-0.07153671, 0.20067534, 0.0070963474, 0.0630...","[1.3940898, 1.8590791, 0.84730095, 1.3343887, ..."
7799,Federal Judge Curbs Enforcement of North Carol...,A federal judge on Friday curbed the enforceme...,"[0.004603547, 0.019374434, -0.011322883, -0.01...","[0.24817775, -2.4326952, -2.1218348, 2.196453,..."
7800,Mexicans Accuse President of ‘Historic Error’ ...,MEXICO CITY — If President Enrique Peña Nie...,"[-0.028931862, 0.0069274977, -0.0069188196, 0....","[0.4804474, 2.163277, 0.16321248, 2.232962, 2...."
7801,"U.S. Presses for Truce in Syria, With Its Larg...","HANGZHOU, China — The image of a Syrian ...","[-0.022897683, 0.3731831, -0.033410747, 0.0183...","[0.65167874, 2.3952546, 1.9429837, 3.6491663, ..."


In [31]:
def cosine_similarity(df):
    return 1 - spatial.distance.cosine(df[0], df[1])

In [32]:
df['d2v_similarity'] = df[['embed_title_d2v', 'embed_content_d2v']].apply(cosine_similarity, axis = 1)
df['d2v_similarity'].mean()

0.37332964408136426

In [33]:
embed_USE = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

def embed_with_USE(embedder , x):
    return embedder(x.split('.')).numpy().mean(axis = 0)

df['embed_title_USE'] = df['title'].apply(lambda x: embed_with_USE(embed_USE, x))
df['embed_content_USE'] = df['content'].apply(lambda x: embed_with_USE(embed_USE, x))
df['USE_similarity'] = df[['embed_title_USE', 'embed_content_USE']].apply(cosine_similarity, axis = 1)
df['USE_similarity'].mean()


0.4937262279795504

In [39]:
model_BERT = SentenceTransformer(r"sentence-transformers/paraphrase-MiniLM-L6-v2")

def embed_with_USE(model , x):
    return model.encode(x)

df['embed_title_BERT'] = df['title'].apply(lambda x: embed_with_USE(model_BERT, x))
df['embed_content_BERT'] = df['content'].apply(lambda x: embed_with_USE(model_BERT, x))
df['BERT_similarity'] = df[['embed_title_BERT', 'embed_content_BERT']].apply(cosine_similarity, axis = 1)
df['BERT_similarity'].mean()

0.5456832346447217

In [42]:
elmo2 = hub.Module("https://tfhub.dev/google/elmo/2")
embeddings = elmo2(
    ["the cat is on the mat", "dogs are in the fog"],
    signature="default",
    as_dict=True)["elmo"]
embeddings

RuntimeError: variable_scope module_1/ was unused but the corresponding name_scope was already taken.